In [1]:
import tensorflow as tf
import keras 
from keras.applications import VGG16
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator

Using TensorFlow backend.


In [2]:
conv_base = VGG16(weights='imagenet', include_top = False, input_shape=(300,300,3))

In [ ]:
# conv_base.summary()

In [3]:
import pickle
X = pickle.load(open(f'pickle_files/X298.pkl', 'rb'))
y = pickle.load(open(f'pickle_files/y298.pkl', 'rb'))
X = X.permute(0,2,3,1).numpy()
y = y.numpy()

dataset_size = X.shape[0]

In [4]:
import torch
inds = torch.randperm(dataset_size)
train_indices = inds[:int(0.8*dataset_size)]
test_indices = inds[int(0.8*dataset_size):]


X_train = X[train_indices]
X_test = X[test_indices]
y_train = y[train_indices]
y_test = y[test_indices]

In [5]:
nos = X_train.shape[0]

In [6]:
datagen = ImageDataGenerator(rescale=1./255)

In [26]:
bs = 34
numit = NumpyArrayIterator(
    X_train, y_train, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

In [27]:
train_samples = np.zeros(shape=(nos,9,9,512))
train_labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    train_samples[i*bs:(i+1)*bs] = f
    train_labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)


In [28]:
train_samples = train_samples.reshape(nos,9*9*512)
train_samples.shape, train_labels.shape

((238, 41472), (238,))

In [29]:
nostest = X_test.shape[0]
nostest

60

In [30]:
numitest = NumpyArrayIterator(
    X_test, y_test, datagen, batch_size=nostest, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(nostest,9,9,512))
test_labels = np.zeros(shape=(nostest))
for samples_batch,labels_batch in numitest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

(60, 9, 9, 512)


In [42]:
test_samples = test_samples.reshape(nostest,9*9*512)
test_samples.shape

(60, 41472)

In [34]:
from keras import models, layers, optimizers
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 2e-5),
              loss = 'binary_crossentropy',
              metrics = ['acc'])
        


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [43]:
history = model.fit(train_samples, train_labels, epochs = 30, batch_size = 20,
                    validation_data = (test_samples, test_labels)
)

Train on 238 samples, validate on 60 samples
Epoch 1/30
238/238 [==============================] - 1s 6ms/step - loss: 0.6795 - acc: 0.6092 - val_loss: 0.5948 - val_acc: 0.6333
Epoch 2/30
238/238 [==============================] - 1s 5ms/step - loss: 0.5739 - acc: 0.7185 - val_loss: 0.5578 - val_acc: 0.6833
Epoch 3/30
238/238 [==============================] - 1s 5ms/step - loss: 0.4614 - acc: 0.8109 - val_loss: 0.4940 - val_acc: 0.8000
Epoch 4/30
238/238 [==============================] - 1s 5ms/step - loss: 0.4491 - acc: 0.7689 - val_loss: 0.4586 - val_acc: 0.7833
Epoch 5/30
238/238 [==============================] - 1s 6ms/step - loss: 0.3877 - acc: 0.8529 - val_loss: 0.4382 - val_acc: 0.7833
Epoch 6/30
238/238 [==============================] - 1s 5ms/step - loss: 0.3755 - acc: 0.8403 - val_loss: 0.4376 - val_acc: 0.8500
Epoch 7/30
238/238 [==============================] - 1s 6ms/step - loss: 0.3433 - acc: 0.8782 - val_loss: 0.3971 - val_acc: 0.8000
Epoch 8/30
238/238 [===========